# Zonal Exchange Network
ZEN（Zonal Exchange Network） 是一个原创的水文模型，用于模拟流域内的水文过程。ZEN使用神经网络来描述产汇流过程。

In [52]:
# 自动加载模块
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. 数据处理

In [53]:
from hydronetwork.data.camels_us_hourly import load_timeseries
from hydronetwork.dataset.preprocessing import split_timeseries
from hydronetwork.dataset.dataset import get_dataset

lookback = 168
horizon = 5
timeseries = load_timeseries(gauge_id='13023000', unit="mm/h")
train_timeseries, test_timeseries = split_timeseries(timeseries, split_list=[0.8, 0.2])
train_dataset = get_dataset(train_timeseries, use_multiprocessing=True, batch_size=512,
                            features_bidirectional=['total_precipitation', 'potential_evaporation'],
                            target="streamflow", lookback=lookback, horizon=horizon)
test_dataset = get_dataset(test_timeseries, use_multiprocessing=True, batch_size=512,
                           features_bidirectional=['total_precipitation', 'potential_evaporation'],
                           target="streamflow", lookback=lookback, horizon=horizon)

## 2. 模型构建

In [54]:
from hydronetwork.model import ZonalExchangeNet

model = ZonalExchangeNet(m=3,
                         n=16,
                         n_mix_steps=2,
                         water_capacity_max=60,
                         layer_units=[16, 8, 1],
                         horizon=horizon, )

## 3. 模型训练

In [ ]:
from hydronetwork.train import WarmupExponentialDecay, callback_for_features_selection
from hydronetwork.evaluate.metric import rmse, nse
from keras.src.optimizers import AdamW

model.compile(optimizer=AdamW(learning_rate=WarmupExponentialDecay(dataset_length=len(train_dataset),
                                                                   initial_learning_rate=1e-5, )),
              loss=rmse,
              metrics=[nse]
              )
model.fit(train_dataset,
          epochs=60,
          verbose=1,
          # shuffle=True,
          # validation_data=test_dataset,
          callbacks=callback_for_features_selection()) 

Epoch 1/60
251/267 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - loss: 0.2317 - nse: 13026.5459 